In [ ]:
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -qU pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 83.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [ ]:
import os

CSVS_DIR_PATH = '/content/drive/MyDrive/CSVs/'
YOUTUBE_HANDLES_DIR_PATH = os.path.join(CSVS_DIR_PATH,'Youtube_handles')
AFTER_DIR_PATH = os.path.join(YOUTUBE_HANDLES_DIR_PATH,'after_preprocessing')

In [ ]:
import pandas as pd

combined_df = pd.read_csv(os.path.join(AFTER_DIR_PATH,'combined_df_with_ids.csv'))

## Test

In [ ]:
import requests
youtube_handles = combined_df['youtube_handle'].unique()
print(len(youtube_handles))
test_handle = youtube_handles[0]
test_handle = "@xerath"
youtube_api_key = userdata.get('youtube_api_key_4')
request_url = f"https://www.googleapis.com/youtube/v3/search?part=snippet&q={test_handle}&type=channel&key={youtube_api_key}"
print(request_url)
# response = requests.get(request_url)
# response_json = response.json()
# print(response_json['items'][0]['id']['channelId'])

7729


SecretNotFoundError: Secret youtube_api_key_4 does not exist.

## Execution

In [ ]:
def sending_request(handle, api):
  request_url = f"https://www.googleapis.com/youtube/v3/search?part=snippet&q={handle}&type=channel&key={api}"
  response = requests.get(request_url)
  return response

In [ ]:
def response_parsing(response_json):
  try:
    channel_id = response_json['items'][0]['id']['channelId']
  except:
    channel_id = None
  return channel_id

In [ ]:
def skip_gen(combined_df_csv, youtube_handle):
  skipping_rows = []
  for i in range(len(combined_df_csv)):
    row = combined_df_csv.iloc[i]
    if (row['channel_id'] is np.nan):
      skipping_rows.append(i)
    if (row['youtube_handle'] == youtube_handle):
      break
  return skipping_rows

In [ ]:
import numpy as np

update_needed = 0
youtube_api_key = userdata.get("youtube_api_key_05")

combined_df = pd.read_csv(os.path.join(AFTER_DIR_PATH,'combined_df_with_ids.csv'))
for i in range(len(combined_df)):
  row = combined_df.iloc[i]
  channel_id = row['channel_id']
  if (channel_id is np.nan):
    update_needed+=1

print("Number of updates required: ",update_needed)

Number of updates required:  180


In [ ]:
skipping_rows = skip_gen(combined_df, "@hanjincherry9996")
print(update_needed - len(skipping_rows))

84


In [ ]:
import requests
updated_count = 0
for i in range(len(combined_df)):
  row = combined_df.iloc[i]
  youtube_handle = row['youtube_handle']
  category = row['category']
  channel_id = row['channel_id']
  if (channel_id is np.nan and i not in skipping_rows):
    response = sending_request(youtube_handle, youtube_api_key)
    new_channel_id = response_parsing(response.json())
    combined_df.at[i, 'channel_id'] = new_channel_id
    if (new_channel_id is not None):
      updated_count+=1

print("Total update: ",updated_count)
print("Saving!")
combined_df.to_csv(os.path.join(AFTER_DIR_PATH,'combined_df_with_ids.csv'), index=False)

Total update:  0
Saving!
